# Yotuubef - YouTube Automation with Enhanced Error Handling

This notebook clones the yotuubef repository, installs dependencies, and runs the script with GPU acceleration (T4) and enhanced error handling for Reddit downloads and YouTube authentication.

## Features
- ✅ Enhanced Reddit video downloading with 403 error mitigation
- ✅ Improved YouTube authentication for Colab
- ✅ Better error handling and retry logic
- ✅ GPU acceleration support

## One-Click Setup

1. **Run all cells** (Runtime > Run all)
2. Upload your `.env` file when prompted
3. The script will automatically handle authentication and processing

> **Note**: Get API keys from:
> - YouTube: https://console.cloud.google.com/apis/credentials
> - Reddit: https://www.reddit.com/prefs/apps
> - Gemini: https://aistudio.google.com/app/apikey

In [ ]:
# Check GPU availability
!nvidia-smi --query-gpu=name --format=csv,noheader
print("GPU check complete!")

In [ ]:
# Clone repository
!git clone https://github.com/beenycool/yotuubef
%cd yotuubef
print("Repository cloned successfully!")

In [ ]:
# Install dependencies
!pip install -r requirements.txt
print("Dependencies installed!")

In [ ]:
# Upload and configure environment
from google.colab import files
import os
import json
import shutil
from datetime import datetime

# Create secure_files directory
if not os.path.exists('secure_files'):
    os.makedirs('secure_files')

print("Please upload your .env file:")
uploaded = files.upload()
env_uploaded = False

for filename in uploaded.keys():
    if filename.endswith('.env'):
        # Move to secure_files
        shutil.move(filename, f'secure_files/{filename}')
        print(f"✅ Uploaded: {filename}")
        env_uploaded = True
        
        # Load environment variables
        with open(f'secure_files/{filename}') as f:
            for line in f:
                if line.strip() and not line.startswith('#') and '=' in line:
                    key, value = line.strip().split('=', 1)
                    os.environ[key] = value.strip('"\'')
        
        # Create YouTube token file from YOUTUBE_TOKEN_JSON if available
        if 'YOUTUBE_TOKEN_JSON' in os.environ:
            try:
                token_data = json.loads(os.environ['YOUTUBE_TOKEN_JSON'])
                with open('secure_files/youtube_token.json', 'w') as f:
                    json.dump(token_data, f, indent=2)
                os.environ['YOUTUBE_TOKEN_FILE'] = 'secure_files/youtube_token.json'
                print("✅ YouTube token configured from environment")
            except Exception as e:
                print(f"⚠️ Error creating YouTube token: {e}")
        
        # Copy .env to root for dotenv
        shutil.copy(f'secure_files/{filename}', '.env')
        print("✅ Environment variables configured")
        break

if not env_uploaded:
    print("❌ Warning: No .env file uploaded. The script may not work properly")
else:
    print("🚀 Setup complete! Ready to run the script.")

In [ ]:
# Test authentication and setup
print("Testing YouTube authentication...")
try:
    # Try the new auth script first
    !python auth_youtube.py
    print("✅ YouTube authentication successful!")
except Exception as auth_error:
    print(f"⚠️ Auth script error: {auth_error}")
    print("Trying fallback authentication...")
    try:
        !python -c "from script import setup_api_clients; reddit_ok, youtube_ok, gemini_ok = setup_api_clients(); print(f'Reddit: {reddit_ok}, YouTube: {youtube_ok}, Gemini: {gemini_ok}')"
        print("✅ Fallback authentication completed!")
    except Exception as fallback_error:
        print(f"❌ Fallback authentication failed: {fallback_error}")

print("\n🔧 Running system diagnostics...")
!python -c "from script import validate_environment, check_ffmpeg_install; validate_environment(); print('FFmpeg:', check_ffmpeg_install('ffmpeg')); print('FFprobe:', check_ffmpeg_install('ffprobe'))"

In [ ]:
# Run main script with enhanced error handling
print("🎬 Starting YouTube automation...")
print("This will continuously process Reddit videos and upload to YouTube.")
print("Press Ctrl+C to stop.\n")

try:
    !python script.py
except KeyboardInterrupt:
    print("\n⏹️ Script stopped by user.")
except Exception as e:
    print(f"\n❌ Script error: {e}")
    print("\n📊 Running diagnostics...")
    !python diagnostic.py
    
print("\n✅ Session complete!")

## Troubleshooting

### Common Issues & Solutions

#### Reddit 403 Blocked Errors
- ✅ **Fixed**: Enhanced downloader with user-agent rotation and retry logic
- The script now automatically retries with different user agents

#### YouTube Authentication Errors
- ✅ **Fixed**: Improved token handling for Colab environment
- Make sure your `.env` file contains valid `YOUTUBE_TOKEN_JSON`

#### Memory Issues
- Use GPU acceleration when available
- The script includes automatic cleanup

#### FFmpeg Not Found
- Colab includes FFmpeg by default
- Check diagnostics output above

### Getting API Keys

1. **YouTube API**:
   - Go to [Google Cloud Console](https://console.cloud.google.com/)
   - Create a project and enable YouTube Data API v3
   - Create OAuth 2.0 credentials

2. **Reddit API**:
   - Go to [Reddit App Preferences](https://www.reddit.com/prefs/apps)
   - Create a new app (script type)
   - Note the client ID and secret

3. **Gemini API**:
   - Go to [Google AI Studio](https://aistudio.google.com/app/apikey)
   - Create a new API key

### .env File Format
```
# Reddit API
REDDIT_CLIENT_ID="your_reddit_client_id"
REDDIT_CLIENT_SECRET="your_reddit_client_secret"
REDDIT_USER_AGENT="python:VideoBot:v1.6 (by /u/YOUR_USERNAME)"

# YouTube API
GOOGLE_CLIENT_SECRETS_FILE="client_secret_xxx.json"
YOUTUBE_TOKEN_FILE="youtube_token.json"
YOUTUBE_TOKEN_JSON='{"token": "...", "refresh_token": "...", ...}'

# Gemini API
GEMINI_API_KEY="your_gemini_api_key"
```